In [1]:
import requests
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from dotenv import load_dotenv
import os


def extract_data():
    url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)
    return df

df = extract_data()
df

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,91996.000000,1835521937338,1,1835521937338,9.588138e+10,95904.000000,...,1.995016e+07,2.100000e+07,126080.000000,-27.13570,2025-10-06T18:57:42.558Z,67.810000,1.353794e+05,2013-07-06T00:00:00.000Z,None,2025-11-17T21:16:14.994Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,3014.610000,363862275590,2,363862275590,4.115094e+10,3215.640000,...,1.206962e+08,NaN,4946.050000,-39.30336,2025-08-24T19:21:03.333Z,0.432979,6.932560e+05,2015-10-20T00:00:00.000Z,"{'times': 42.8070348266137, 'currency': 'btc',...",2025-11-17T21:16:14.808Z
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,0.999016,183858282266,3,189319806849,1.382085e+11,0.999822,...,1.895074e+11,NaN,1.320000,-24.49449,2018-07-24T00:00:00.000Z,0.572521,7.449313e+01,2015-03-02T00:00:00.000Z,None,2025-11-17T21:16:15.521Z
3,ripple,xrp,XRP,https://coin-images.coingecko.com/coins/images...,2.150000,129249929027,4,214745212640,5.909953e+09,2.280000,...,9.998576e+10,1.000000e+11,3.650000,-41.33317,2025-07-18T03:40:53.808Z,0.002686,7.953700e+04,2014-05-22T00:00:00.000Z,None,2025-11-17T21:16:13.791Z
4,binancecoin,bnb,BNB,https://coin-images.coingecko.com/coins/images...,902.730000,124327456062,5,124327456062,3.266149e+09,937.110000,...,1.377369e+08,2.000000e+08,1369.990000,-34.20849,2025-10-13T08:41:24.131Z,0.039818,2.263562e+06,2017-10-19T00:00:00.000Z,None,2025-11-17T21:16:14.759Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ignition-fbtc,fbtc,Function FBTC,https://coin-images.coingecko.com/coins/images...,92344.000000,1107901687,96,1107901687,5.042780e+05,97489.000000,...,1.194764e+04,NaN,127954.000000,-27.52894,2025-10-06T12:50:40.428Z,11469.830000,7.084664e+02,2024-09-10T18:18:02.102Z,None,2025-11-17T21:16:09.576Z
96,syrupusdt,syrupusdt,syrupUSDT,https://coin-images.coingecko.com/coins/images...,1.100000,1099595073,97,1099595073,4.208077e+06,1.100000,...,9.988401e+08,NaN,1.110000,-0.48016,2025-10-10T21:40:27.453Z,1.093000,6.850900e-01,2025-09-30T09:40:48.362Z,None,2025-11-17T21:16:14.449Z
97,ripple-usd,rlusd,Ripple USD,https://coin-images.coingecko.com/coins/images...,0.999970,1088856892,98,1088856892,1.389340e+08,1.001000,...,1.089198e+09,NaN,1.073000,-6.82630,2024-12-26T10:45:52.337Z,0.962292,3.875370e+00,2024-12-18T04:40:40.880Z,None,2025-11-17T21:16:13.290Z
98,flare-networks,flr,Flare,https://coin-images.coingecko.com/coins/images...,0.013737,1078637624,99,1434976956,8.177239e+06,0.014462,...,1.043327e+11,NaN,0.150073,-90.84931,2023-01-10T03:14:05.921Z,0.008274,6.597321e+01,2023-10-19T03:35:41.663Z,None,2025-11-17T21:16:08.616Z


In [2]:
load_dotenv()

# Check if .env file is loaded
print("=== Environment Variables Debug ===")
print(f"DB_HOST: {os.getenv('DB_HOST')}")
print(f"DB_NAME: {os.getenv('DB_NAME')}")
print(f"DB_USER: {os.getenv('DB_USER')}")
print(f"DB_PORT: {os.getenv('DB_PORT')}")
print(f"DB_PASS exists: {os.getenv('DB_PASS') is not None}")
print(f"DB_PASS length: {len(os.getenv('DB_PASS') or '')}")

=== Environment Variables Debug ===
DB_HOST: localhost
DB_NAME: etls
DB_USER: postgres
DB_PORT: 5432
DB_PASS exists: True
DB_PASS length: 12


In [3]:
def transform_data(df):
    
    df = df[['id', 'symbol', 'name', 'current_price', 'market_cap', 'total_volume', 'last_updated']]
    
    df = df.dropna()
    
    df['loaded_at'] = pd.Timestamp.now()
    
    return df


transform_data(df) 

,id,symbol,name,current_price,market_cap,total_volume,last_updated,loaded_at
0,bitcoin,btc,Bitcoin,91996.000000,1835521937338,9.588138e+10,2025-11-17T21:16:14.994Z,2025-11-18 04:16:59.919671
1,ethereum,eth,Ethereum,3014.610000,363862275590,4.115094e+10,2025-11-17T21:16:14.808Z,2025-11-18 04:16:59.919671
2,tether,usdt,Tether,0.999016,183858282266,1.382085e+11,2025-11-17T21:16:15.521Z,2025-11-18 04:16:59.919671
3,ripple,xrp,XRP,2.150000,129249929027,5.909953e+09,2025-11-17T21:16:13.791Z,2025-11-18 04:16:59.919671
4,binancecoin,bnb,BNB,902.730000,124327456062,3.266149e+09,2025-11-17T21:16:14.759Z,2025-11-18 04:16:59.919671
...,...,...,...,...,...,...,...,...
95,ignition-fbtc,fbtc,Function FBTC,92344.000000,1107901687,5.042780e+05,2025-11-17T21:16:09.576Z,2025-11-18 04:16:59.919671
96,syrupusdt,syrupusdt,syrupUSDT,1.100000,1099595073,4.208077e+06,2025-11-17T21:16:14.449Z,2025-11-18 04:16:59.919671
97,ripple-usd,rlusd,Ripple USD,0.999970,1088856892,1.389340e+08,2025-11-17T21:16:13.290Z,2025-11-18 04:16:59.919671
98,flare-networks,flr,Flare,0.013737,1078637624,8.177239e+06,2025-11-17T21:16:08.616Z,2025-11-18 04:16:59.919671


In [ ]:
load_dotenv() 

DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_PORT = os.getenv("DB_PORT")

def load_pgsql(df):
    try:
        conn = psycopg2.connect(
            host = DB_HOST,
            dbname = DB_NAME,
            user = DB_USER,
            password = DB_PASS,
            port = DB_PORT
        )
        
        curr = conn.cursor()

        table_query = """ 
        CREATE TABLE IF NOT EXISTS crypto_price (
            id TEXT, 
            symbol TEXT, 
            name TEXT, 
            current_price DECIMAL(20, 2), 
            market_cap BIGINT, 
            total_volume BIGINT, 
            last_updated TIMESTAMP,
            loaded_at TIMESTAMP
        ) 
        """
        
        curr.execute(table_query)
        conn.commit()
        
        insert = """
            INSERT INTO crypto_price (id, symbol, name, current_price, market_cap, total_volume, last_updated, loaded_at) VALUES %s
        """
        
        execute_values(curr, insert, df.values.tolist())
        conn.commit()
        
        curr.close()
        conn.close()
        
        print("Data loaded successfully!")
        
    except Exception as e:
        print(f"Error loading: {e}")
        if 'conn' in locals():
            conn.rollback()
            conn.close()

load_pgsql(df)

Error loading data: can't adapt type 'dict'
